# AI Engineering Drawing Inspector v2.0

**Single-File QC Pipeline**

Outputs:
1. `ResolvedPartIdentity.json`
2. `DrawingEvidence.json`
3. `DiffResult.json`
4. `QCReport.md`

In [ ]:
# Cell 1: Install Dependencies
!pip install -q pymupdf opencv-python-headless jsonschema pillow pytesseract
!pip install -q accelerate qwen-vl-utils bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers
!apt-get install -y poppler-utils tesseract-ocr > /dev/null 2>&1
print("Dependencies installed!")